In [18]:
import pandas as pd
import numpy as np

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [29]:
def scored_news(COMPANY):
    finwiz_url = 'https://finviz.com/quote.ashx?t='
    
    news_tables = {}
    tickers = [COMPANY]

    for ticker in tickers:
        url = finwiz_url + ticker
        req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
        response = urlopen(req)    
        html = BeautifulSoup(response)
        news_table = html.find(id='news-table')
        news_tables[ticker] = news_table

    news = news_tables[COMPANY]
    news_tr = news.findAll('tr')

    for i, table_row in enumerate(news_tr):
        a_text = table_row.a.text
        td_text = table_row.td.text
        if i == 7:
            break    

    parsed_news = []

    for file_name, news_table in news_tables.items():
        for x in news_table.findAll('tr'):
            text = x.a.get_text() 
            date_scrape = x.td.text.split()
            if len(date_scrape) == 1:
                time = date_scrape[0]
            else:
                date = date_scrape[0]
                time = date_scrape[1]
            ticker = file_name.split('_')[0]
            parsed_news.append([ticker, date, time, text])

    parsed_news

    vader = SentimentIntensityAnalyzer()

    columns = ['ticker', 'date', 'time', 'headline']
    parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)
    scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()
    scores_df = pd.DataFrame(scores)
    parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')
    parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date
    parsed_and_scored_news.head()
    
    plt.rcParams['figure.figsize'] = [14, 8]
    mean_scores = parsed_and_scored_news.groupby(['ticker','date']).mean()
    mean_scores = mean_scores.unstack()
    mean_scores = mean_scores.xs('compound', axis="columns").transpose()

    fig, ax = plt.subplots()

    ax.bar(mean_scores.index, mean_scores[f'{COMPANY}'], width = 4 )

    plt.grid(color='black', which='major', linewidth=0.5)
    plt.minorticks_on()
    plt.grid(color='black', linestyle = ':', linewidth=0.3, which='minor')
    plt.show()

#     plt.rcParams['figure.figsize'] = [14, 8]
#     mean_scores = parsed_and_scored_news.groupby(['ticker','date']).mean()
#     mean_scores = mean_scores.unstack()
#     mean_scores = mean_scores.xs('compound', axis="columns").transpose()

    return plt.show()